In [3]:
#Import dependencies
import pandas as pd
import requests
import json
from config import cost_api_key
#api_url = 

In [ ]:
#Scrape wikipedia page for top 150 US cities by population


In [ ]:
api_url = "https://cities-cost-of-living1.p.rapidapi.com/get_cities_details"

data = [{}]

payload = "cities=%5B%7B%22name%22%3A%22Lisbon%22%2C%22country%22%3A%22Portugal%22%7D%2C%7B%22name%22%3A%22Paris%22%2C%22country%22%3A%22Spain%20(wrong%20country)%22%7D%2C%7B%22name%22%3A%22Hamilton%22%2C%22country%22%3A%22Bermuda%22%7D%2C%7B%22name%22%3A%22Saint%20Francisc%20(wrong%20city%20name)%22%2C%22country%22%3A%22United%20States%22%7D%5D&currencies=%5B%22Euro%20(wrong%20currency%20code)%22%2C%22USD%22%2C%22CHF%22%5D"
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'x-rapidapi-host': "cities-cost-of-living1.p.rapidapi.com",
    'x-rapidapi-key': cost_api_key
    }

response = requests.request("POST", api_url, data=payload, headers=headers)

print(response.text)

[{"name":"Lisbon","country":"Portugal"},{"name":"Paris","country":"Spain (wrong country)"},{"name":"Hamilton","country":"Bermuda"},{"name":"Saint Francisc (wrong city name)","country":"United States"}]